In [33]:
import geohash
import pandas as pd
import configparser
import googlemaps
import time
import json
import os
import networkx as nx
from networkx.readwrite import json_graph

In [9]:
BASE_32 = "0123456789bcdefghjkmnpqrstuvwxyz"

NEIGHBORS = {'right':{'even': "bc01fg45238967deuvhjyznpkmstqrwx",'odd':"238967debc01fg45kmstqrwxuvhjyznp"},
             'left':{'even':"238967debc01fg45kmstqrwxuvhjyznp",'odd':"14365h7k9dcfesgujnmqp0r2twvyx8zb"},
             'top':{'even':"p0r21436x8zb9dcf5h7kjnmqesgutwvy",'odd':"bc01fg45238967deuvhjyznpkmstqrwx"},
             'bottom':{'even':"14365h7k9dcfesgujnmqp0r2twvyx8zb",'odd':"238967debc01fg45kmstqrwxuvhjyznp"}
            }
             
BORDERS = {'right':{'even':"bcfguvyz",'odd':"prxz"},
           'left':{'even':"0145hjnp",'odd':"028b"},
           'top':{'even':"prxz",'odd':"bcfguvyz"},
           'bottom':{'even':"028b",'odd':"0145hjnp"}
          }

def calc_adj_geohash(src_hash, search_direction):
    L_src_hash = src_hash.lower()
    last_char = L_src_hash[-1]
    hash_type = odd_even(L_src_hash)
    base = L_src_hash[0:len(L_src_hash)-1]
    if  last_char in BORDERS[search_direction][hash_type]:
        base = calc_adj_geohash(base, search_direction)
    return base + BASE_32[NEIGHBORS[search_direction][hash_type].index(last_char)]

In [10]:
def get_adj_geohashes(src_hash):
    directions = ['right','left','top','bottom']
    adj_hash_dict = {direction:calc_adj_geohash(src_hash,direction) for direction in directions}
    adj_hash_dict['top_right'] = calc_adj_geohash(adj_hash_dict['right'],'top')
    adj_hash_dict['bottom_right'] = calc_adj_geohash(adj_hash_dict['right'],'bottom')
    adj_hash_dict['top_left'] = calc_adj_geohash(adj_hash_dict['left'],'top')
    adj_hash_dict['bottom_left'] = calc_adj_geohash(adj_hash_dict['left'],'bottom')
    adj_hash_dict['middle'] = src_hash
    return adj_hash_dict    

In [11]:
def odd_even(str_hash):
    if len(str_hash)%2 == 0:
        return 'even'
    else:
        return 'odd'

In [12]:
config = configparser.ConfigParser()
config.read("config.ini")
API_key = config['Keys']['google_API']
gmaps = googlemaps.Client(key=API_key)

def get_geohash_distance(gh_A,gh_B):   
    GPS_A = geohash.decode(gh_A)
    GPS_B = geohash.decode(gh_B) 
    directions_result = gmaps.directions(GPS_A,
                                         GPS_B,
                                         mode="driving")
    time.sleep(1)
    return directions_result[0]['legs'][0]['distance']['value'] 

In [59]:
GEOHASH_PRECISION = 3
MAX_RANGE = 426 #Maximum Tesla Model S range in Km from https://en.wikipedia.org/wiki/Tesla_Model_S
def build_connections(G,src_hash):
    print (nx.number_of_nodes(G))
    connections = {}
    node_hashes = ([node for node in G
                  if node[0:GEOHASH_PRECISION] in list(get_adj_geohashes(src_hash[0:GEOHASH_PRECISION]).values())
                  and node != src_hash])
    close_connections = ([{'node':node_gh,
                           'distance':get_geohash_distance(src_hash,node_gh)} for node_gh in node_hashes])
    for connection in close_connections:
        if connection['distance']/1000 <= MAX_RANGE:
            G.add_edge(src_hash,connection['node'],{'distance':connection['distance'],
                                                    'lon_lat_1':reverse_GPS(geohash.decode(src_hash)),
                                                    'lon_lat_2':reverse_GPS(geohash.decode(connection['node']))})
    return G

In [37]:
def reverse_GPS(GPS):
    return [GPS[1],GPS[0]]

In [67]:
def build_network():
    G=nx.Graph()
    df = pd.read_csv("Teslarati_SC_data.csv")
    df["lat"], df["lon"] = zip(*df["GPS"].str.split(',').tolist())
    df["lat"], df["lon"] = df["lat"].astype(float), df["lon"].astype(float)
    df['GPS_lon_lat'] = df.apply(lambda x: [x["lon"],x["lat"]], axis=1)
    df['geohash'] = df.apply(lambda x: geohash.encode(x['lat'],x['lon']), axis=1)
    
    for i in df['geohash'].keys():
        G.add_node(df['geohash'][i],{'GPS_lon_lat':df['GPS_lon_lat'][i]})
        build_connections(G,df['geohash'][i])
    
    data = json_graph.node_link_data(G)
    print (data)
    with open("connections.json","w") as f:
        json.dump(data,f)
    return G

In [ ]:
test_G = build_network()

In [32]:
test_G.edges()

[('dr4qbsffydeg', 'dr6pbb658ut2'),
 ('dr4qbsffydeg', 'dr63w0p6cwbt'),
 ('dr63w0p6cwbt', 'dr6pbb658ut2'),
 ('9qeqhugxq5w5', '9qd9x3ey0frp'),
 ('9qd9x3ey0frp', '9q717h3ht9ye'),
 ('dnm60k31rgq5', 'dnjqp5cgs92e'),
 ('dnxx0b8z7584', 'dnw65tcbz7q7'),
 ('9q717h3ht9ye', '9q5f7vrfnun9')]